# Import Required Packages

In [1]:
# Imports
import os
import datetime
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time



# Input data from User

In [2]:
#Market analysed: 'Investment','FullYear','DayAhead','Balancing' (choose one or several)
market_analysed=['DayAhead','Balancing'] 
output='CurtailmentHourly'
first_timestep="2012-01-02"
#Number of timesteps (total number of combination of SSS and TTT)
number_periods=8736*12 
#Time size of each time step for creating timestamp
size_timestep="300s"
#Time size of each TTT calculating energy values
size_t=1/12;




# Plot Settings

In [3]:
# Set plotting specifications
% matplotlib inline
plt.rcParams.update({'font.size': 21})
plt.rcParams['xtick.major.pad']='12'
plt.rc('legend', fontsize=16)
y_limit = 1.1
lw = 3

# Read Input Files

In [4]:
data=pd.DataFrame()
for market in market_analysed:
    csvfiles = []
    for file in glob.glob("./input/results/" + market + "/*.csv"):
        csvfiles.append(file)

    csvfiles=[file.replace('./input\\','') for file in csvfiles] 
    csvfiles=[file.replace('.csv','') for file in csvfiles]  
    csvfiles=[file.split('_') for file in csvfiles]  
    csvfiles = np.asarray(csvfiles)  
    csvfiles=pd.DataFrame.from_records(csvfiles)
    
    csvfiles.rename(columns={0: 'Output', 1: 'Scenario',2: 'Year',3:'Subset'}, inplace=True)
    scenarios=csvfiles.Scenario.unique().tolist()
    years=csvfiles.Year.unique().tolist()
    subsets=csvfiles.Subset.unique().tolist()

    for scenario in scenarios:
        for year in years:
            for subset in subsets:
                file = "./input/results/"+ market + "/"+ output + "_" + scenario + "_" + year + "_" + subset + ".csv"
                if os.path.isfile(file):
                    df=pd.read_csv(file,encoding='utf8')
                    df['Scenario'] = scenario
                    df['Market']   = market
                    data=data.append(df) 

                
                

In [ ]:
for market in market_timestep_proc:
    df_copy=data.loc[data.Market == market]

In [5]:
#Converting timesteps
if SSS_TTT_consistent==0:
    data['SSS_NEW']=data['SSS']+data['TTT']
    data['TTT_NEW']=data['SSS']+data['TTT']
    dict_timesteps_conversion_SSS=pd.read_csv('./input/timesteps_conversion_SSS.csv', index_col=0, squeeze=True).to_dict()
    dict_timesteps_conversion_TTT=pd.read_csv('./input/timesteps_conversion_TTT.csv', index_col=0, squeeze=True).to_dict()
# and replace in the corresponding column
    data['SSS_NEW']=data['SSS_NEW'].map(dict_timesteps_conversion_SSS)
    data['TTT_NEW']=data['TTT_NEW'].map(dict_timesteps_conversion_TTT)
    data.SSS_NEW.fillna(data.SSS, inplace = True) 
    data.TTT_NEW.fillna(data.TTT, inplace = True) 
# and cleaning data
    del data['SSS']
    del data['TTT']                                 
    data.rename({'SSS_NEW': 'SSS', 'TTT_NEW': 'TTT'}, axis=1, inplace=True) 


 

In [6]:
#Timestamp addition
full_timesteps = pd.read_csv('./input/full_timesteps.csv')
full_timesteps.Key=full_timesteps['SSS']+full_timesteps['TTT']
full_timesteps['timestamp']= pd.date_range(first_timestep, periods = number_periods, freq =size_timestep)
dict_timestamp=dict(zip(full_timesteps.Key, full_timesteps.timestamp))
data['timestamp']=data['SSS']+data['TTT']
data['timestamp']=data['timestamp'].map(dict_timestamp)



C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#Copying and appending as many values as elements in TTT
for market in market_timestep_proc:

# Additional set declaration

In [ ]:
ccc = list(data.C.unique())
rrr = list(data.RRR.unique())
tech_type = list(data.TECH_TYPE.unique())
commodity = list(data.COMMODITY.unique())
fff = list(data.FFF.unique())
sss = list(full_timesteps.SSS.unique())
ttt = list(full_timesteps.TTT.unique())


# Time step selection

In [ ]:
# Seasons to investigate
# season_names = ['S01', 'S07', 'S20', 'S24', 'S28', 'S38', 'S42', 'S43']
# Make a list of every nth element of sss (1 <= nth <= number of elements in sss)
nth = 1
s = sss[0::nth]
 # Or select seasons by names
# s = season_names


In [ ]:
# Terms to investigate
# term_names = ['T005', 'T019', 'T033', 'T047', 'T061', 'T075', 'T089', 'T103', 'T117', 'T131', 'T145', 'T159']
# Make a list of every nth element of ttt (1 <= nth <= number of elements in ttt)
nth = 1
t = ttt[0::nth]
# Or select terms by name
# t = term_names

# Make Directories


In [ ]:
# Make output folder
if not os.path.isdir('output'):
    os.makedirs('output')

In [ ]:
# Make CurtailmentHourly folder
if not os.path.isdir('output/' + output):
    os.makedirs('output/' + output)

In [ ]:
# Make market folder
for market in market_analysed:
    if not os.path.isdir('output/' + output + '/'+ market +'/Country_wise'):
        os.makedirs('output/' + output + '/'+ market +'/Country_wise')
# Make country folder
    if not os.path.isdir('output/' + output  + '/'+ market +'/Country_wise'):
        os.makedirs('output/' + output  + '/'+ market  +'/Country_wise')
    # Make country wise folders
    for c in ccc:
        if not os.path.isdir('output/' + output  + '/'+ market +'/Country_wise/' + c):
            os.makedirs('output/' + output  + '/'+ market +'/Country_wise/' + c)

# Plotting

In [ ]:
# Make data frames to plot
data_plot = data[(data.SSS.isin(s)) & (data.TTT.isin(t))]


## Plot per year 

In [ ]:
for i in years:
    spp_plot[data.SSS.isin([i])][ccc[:2]].plot(figsize=(16,9), lw=lw)
    plt.ylim([0, y_limit])
    plt.legend(loc=1)
    plt.title('Curtailment in ' +  i)
    plt.xlabel('Terms')
    plt.xticks(t_marker, t_selected, rotation=45)
    for x_pos in t_marker:
        plt.axvline(x=x_pos, c='black', lw=6, alpha=0.3)
    plt.tight_layout()
    plt.savefig('output/pv_production/spp_' + i + '.png', compression=None)
    # plt.show()
    plt.close()
plt.close()

In [ ]:
#Plot example with several x axis

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()

# Add some extra space for the second axis at the bottom
fig.subplots_adjust(bottom=0.2)
ax1.set_xticks([1,2,4,5,7,8])
ax1.set_xlim(0,9)
ax1.set_xticklabels(('2015','2016','2015','2016','2015','2016'))
ax2.spines["bottom"].set_position(("axes", -0.15))
ax2.xaxis.set_ticks_position("bottom")
ax2.spines["bottom"].set_visible(True)
ax2.set_xticks([1.5,4.5,7.5])
ax2.set_xticklabels(('1','2','3'))
ax2.set_xlim(0,9)

b1 = np.random.randint(0,100,6)
b2 = np.random.randint(0,100,6)
b3 = np.random.randint(0,100,6)
plt.bar(np.array([1,2,4,5,7,8])-0.4,b1,color='blue')
plt.bar(np.array([1,2,4,5,7,8])-0.4,b2,color='orange',bottom=b1)
plt.bar(np.array([1,2,4,5,7,8])-0.4,b3,color='yellow',bottom=b1+b2)

plt.show()